In [7]:
import pandas as pd
import numpy as np
import sys
import os

In [8]:
data = '../../Output'
          
files = []
for folder in os.listdir(data):
    if folder != '.ipynb_checkpoints' and folder != 'OutputStats.txt' and folder != 'FirmNameStats.txt':
        for file in os.listdir(os.path.join(data, folder)):
            files.append((pd.read_csv(os.path.join(os.path.join(data, folder), file)), folder + '/' + file))

In [9]:
firms_df = pd.read_excel('../Data/FirmNameData/cleaned_firms.xlsx')
all_firms = [str(firm).lower() for firm in firms_df['Name'].tolist()]

In [10]:
lop = []
for f in files:
    loe = []
    count = 0
    for ent in f[0]['Hits']:
        for e in str(ent).split(','):
            if e.lstrip() not in loe:
                loe.append(e.lstrip())
    for fi in all_firms:
        if fi in loe:
            count += 1
    lop.append(int(count/len(all_firms) * 100))

# Stats dictionary
stats_dict = {}
for f in files:
    stats_dict[f[1].split('/')[0]] = {'Percentage of text entries that had >= 1 hit': 0, 'Percentage of crsp firms hit': 0}

record = open('../../Output/OutputStats.txt', 'w')
i = 0
record.write('-------------------------------MATCH DATA FOR EACH FILE-------------------------------\n \n')
for f in files:
    p_hits = int(len(f[0].loc[f[0]['Hits'] != 'No Hits'])/len(f[0]) * 100)
    record.write('File: {}, \nPercentage of text entries that had >= 1 hit: ~{}%, \nPercentage of crsp firms hit: {}% \n \n'.format(
         f[1], p_hits, lop[i]))
    stats_dict[f[1].split('/')[0]]['Percentage of text entries that had >= 1 hit'] += p_hits
    stats_dict[f[1].split('/')[0]]['Percentage of crsp firms hit'] += lop[i]
    i += 1
record.close()

In [11]:
r = open('../../Output/OutputStats.txt', 'a')
r.write('-------------------------------AVERAGES FOR EACH FOLDER-------------------------------\n \n')
for key, val in stats_dict.items():
    r.write('Media Network: {} \n'.format(key))
    for k, v in val.items():
        r.write('{}: {}% \n'.format(k, int(v/9)))
    r.write('\n')
r.close()

In [14]:
f_file = open('../../Output/FirmNameStats.txt', 'w')
ulon = []
for f in files:
    for ent in f[0]['Hits']:
        for e in str(ent).split(', '):
            if e.lstrip() not in ulon:
                ulon.append(e.lstrip())
f_file.write('-------------------------------UNIQUE FIRM NAMES-------------------------------\n \n')
f_file.write(', '.join(ulon))
f_file.write('-------------------------------PERCENTAGE OF CRSP-------------------------------\n \n')
f_file.write('Percentage of crsp firms matched: {}%'.format((len(ulon)/len(all_firms)) * 100))
f_file.close()